In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
df = pd.read_csv("../processed_data/merged_data.csv")

In [ ]:
df2 = pd.read_csv("../processed_data/subgroup_probabilities.csv", index_col=0)
df2['max_value'] = df2[['prob_sub1', 'prob_sub2', 'prob_sub3', 'prob_sub4', 'prob_sub5', 'prob_sub6', 'prob_sub7']].max(axis=1)*100

In [ ]:
df = df.merge(df2[['hadm_id', 'max_value']], how='left', on='hadm_id')

In [ ]:
def convert_int64(dataframe):
    for col in dataframe.columns:
        if dataframe[col].dtype == np.int64:
            dataframe[col] = dataframe[col].astype(int)
    return df
df = convert_int64(df)

Functions to convert to int and get latest vitals

In [ ]:
def get_latest_vitals(dataframe):
    dataframe['charttime'] = pd.to_datetime(dataframe['charttime'])
    dataframe_sorted = dataframe.sort_values(by=['subject_id', 'charttime'])
    latest_vitals = dataframe_sorted.groupby('subject_id').last().reset_index()
    return latest_vitals

latest_vitals_df = get_latest_vitals(df)

latest_vitals_df.head()

Function to convert data to json for website

In [ ]:
def json_converter(dataframe, patient_id):
    data = {
        "name": "John Doe",
    }
    row = dataframe[dataframe['subject_id']==patient_id]
    
    elix = row.iloc[:, -31:-1]
    one_hot_columns = elix.columns[elix.eq(1).any()].tolist()
    one_hot_columns = [' '.join(word.capitalize() for word in item.replace('_', ' ').split()) for item in one_hot_columns]

    def custom_serializer(obj):
        if isinstance(obj, np.integer):
            return int(obj)  # Convert NumPy integers to Python int
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()  # Convert Timestamp to ISO 8601 string
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")
    
    data["subject_id"] = row["subject_id"].iloc[0]
    data["subgroup"] = row["subgroup_x"].iloc[0]
    data["latest_blood_pressure"] = row["Combined_Blood_Pressure"].iloc[0]
    data["latest_chart_time"] = row["charttime"].iloc[0]
    data["latest_temp_F"] = round(row["Final_Temperature_F"].iloc[0], 2)
    data["latest_heart_rate"] = row["heart_rate"].iloc[0]
    data["latest_respiratory_rate"] = row["RespiratoryRate_combined"].iloc[0]
    data["elixhauser"] = one_hot_columns
    data["age"] = row["rounded_age"].iloc[0]
    data["spo2"] = row["SpO2"].iloc[0]
    data["sofa"] = row["sofa"].iloc[0]
    temp1=row["max_value"].iloc[0]
    if temp1==100.0:
        temp1=99.0
    data["prob"] = temp1
    data["white_blood"] = row["WBC_combined"].iloc[0]*1000
    
    file_name = "processed_data/id" + str(patient_id) + ".json"

    with open(file_name, 'w') as json_file:
        json.dump(data, json_file, indent=4, default=custom_serializer)

Combining with wbcc

In [ ]:
df3 = pd.read_csv("processed_data/white_bloodcells_with_sofa.csv", index_col=0)
df3.head()

In [ ]:
white = get_latest_vitals(df3)

In [ ]:
final_df = pd.merge(latest_vitals_df, white[['subject_id', 'WBC_combined']], on='subject_id', how='left')

In [ ]:
json_converter(final_df, 453)

All patient ids sorted

In [ ]:
numbers = [
    26494, 6702, 11167, 3868, 8566, 6659, 17797, 1113,
    12613, 25699, 17807, 7354, 11952, 17795, 22202, 3011,
    6873, 453, 16847, 145, 9467
]

# Sorting the list
sorted_numbers = sorted(numbers)

sorted_numbers